# Revenue vs Expenditure Trends Analysis

This notebook analyzes revenue and expenditure trends from Carmen de Areco's transparency portal.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
from pathlib import Path
import glob

# Set up data paths
data_paths = [
    Path("../data/processed"),
    Path("../data/consolidated"),
    Path("../data/cleaned"),
    Path("../frontend/public/data/csv"),
    Path("../../data/processed"),
    Path("../../data/consolidated"),
    Path("../../frontend/public/data/charts")
]

# Find all revenue and expenditure related CSV files
all_revenue_files = []
all_expenditure_files = []
for path in data_paths:
    if path.exists():
        # Revenue files
        all_revenue_files.extend(list(path.rglob("*revenue*csv")))
        all_revenue_files.extend(list(path.rglob("*ingresos*csv")))
        all_revenue_files.extend(list(path.rglob("*recursos*csv")))
        
        # Expenditure files
        all_expenditure_files.extend(list(path.rglob("*expenditure*csv")))
        all_expenditure_files.extend(list(path.rglob("*gastos*csv")))
        all_expenditure_files.extend(list(path.rglob("*ejecucion*csv")))

print(f"Found {len(all_revenue_files)} revenue files")
print(f"Found {len(all_expenditure_files)} expenditure files")

In [ ]:
# Load the most relevant revenue and expenditure data
revenue_df = None
expenditure_df = None

# Look for consolidated revenue data first
for file_path in all_revenue_files:
    if 'consolidated' in str(file_path) or 'Revenue' in str(file_path) or 'revenue' in str(file_path).lower():
        try:
            revenue_df = pd.read_csv(file_path)
            print(f"Loaded revenue data from: {file_path}")
            break
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            continue

# Look for consolidated expenditure data first
for file_path in all_expenditure_files:
    if 'consolidated' in str(file_path) or 'Expenditure' in str(file_path) or 'expenditure' in str(file_path).lower():
        try:
            expenditure_df = pd.read_csv(file_path)
            print(f"Loaded expenditure data from: {file_path}")
            break
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            continue

# If no consolidated files found, use the first available files
if revenue_df is None and all_revenue_files:
    try:
        revenue_df = pd.read_csv(all_revenue_files[0])
        print(f"Loaded revenue data from: {all_revenue_files[0]}")
    except Exception as e:
        print(f"Error loading {all_revenue_files[0]}: {e}")

if expenditure_df is None and all_expenditure_files:
    try:
        expenditure_df = pd.read_csv(all_expenditure_files[0])
        print(f"Loaded expenditure data from: {all_expenditure_files[0]}")
    except Exception as e:
        print(f"Error loading {all_expenditure_files[0]}: {e}")

# If no files found, create sample data
if revenue_df is None:
    print("No revenue data found, creating sample data for demonstration")
    
    # Sample revenue data
    revenue_data = {
        'year': [2019, 2020, 2021, 2022, 2023],
        'revenue': [2850000, 3100000, 3250000, 3400000, 3550000],
        'source': ['Tax', 'Tax', 'Tax', 'Tax', 'Tax']
    }
    revenue_df = pd.DataFrame(revenue_data)
    
if expenditure_df is None:
    print("No expenditure data found, creating sample data for demonstration")
    
    # Sample expenditure data
    expenditure_data = {
        'year': [2019, 2020, 2021, 2022, 2023],
        'expenditure': [2800000, 3000000, 3200000, 3350000, 3500000],
        'sector': ['General', 'General', 'General', 'General', 'General']
    }
    expenditure_df = pd.DataFrame(expenditure_data)

# Standardize column names
revenue_column_mapping = {
    'Evolucion de los Recursos': 'revenue',
    'Vigente': 'budget',
    'Devengado': 'actual',
    'Percibido': 'received',
    'Año': 'year',
    'anio': 'year',
    'Fuente': 'source',
    'Origen': 'source',
    'Monto': 'revenue'
}

expenditure_column_mapping = {
    'Evolucion de Gastos': 'expenditure',
    'Vigente': 'budget',
    'Preventivo': 'preventive',
    'Compromiso': 'commitment',
    'Devengado': 'actual',
    'Pagado': 'paid',
    'Año': 'year',
    'anio': 'year',
    'Sector': 'sector',
    'Area': 'sector',
    'Departamento': 'sector',
    'Monto': 'expenditure'
}

revenue_df = revenue_df.rename(columns=revenue_column_mapping)
expenditure_df = expenditure_df.rename(columns=expenditure_column_mapping)

# Convert columns to appropriate types
revenue_numeric_cols = ['revenue', 'budget', 'actual', 'received']
for col in revenue_numeric_cols:
    if col in revenue_df.columns:
        revenue_df[col] = pd.to_numeric(revenue_df[col], errors='coerce')

expenditure_numeric_cols = ['expenditure', 'budget', 'preventive', 'commitment', 'actual', 'paid']
for col in expenditure_numeric_cols:
    if col in expenditure_df.columns:
        expenditure_df[col] = pd.to_numeric(expenditure_df[col], errors='coerce')

if 'year' in revenue_df.columns:
    revenue_df['year'] = pd.to_numeric(revenue_df['year'], errors='coerce').astype('Int64')
    
if 'year' in expenditure_df.columns:
    expenditure_df['year'] = pd.to_numeric(expenditure_df['year'], errors='coerce').astype('Int64')

print(f"Revenue dataset shape: {revenue_df.shape}")
print(f"Expenditure dataset shape: {expenditure_df.shape}")

In [ ]:
# Prepare data for visualization
# Merge revenue and expenditure data on year if both exist
merged_df = None

if 'year' in revenue_df.columns and 'year' in expenditure_df.columns:
    # Group by year to aggregate data
    revenue_by_year = revenue_df.groupby('year')['revenue'].sum().reset_index()
    expenditure_by_year = expenditure_df.groupby('year')['expenditure'].sum().reset_index()
    
    # Merge on year
    merged_df = pd.merge(revenue_by_year, expenditure_by_year, on='year', how='outer')
    print("Data merged successfully")
else:
    print("Year column not found in both datasets, using separate data")
    merged_df = pd.DataFrame({
        'year': list(range(2019, 2024)),
        'revenue': revenue_df['revenue'] if 'revenue' in revenue_df.columns else [0, 0, 0, 0, 0],
        'expenditure': expenditure_df['expenditure'] if 'expenditure' in expenditure_df.columns else [0, 0, 0, 0, 0]
    })

# Calculate surplus/deficit
merged_df['balance'] = merged_df['revenue'] - merged_df['expenditure']
merged_df['surplus_deficit'] = merged_df['balance'].apply(lambda x: 'Surplus' if x > 0 else 'Deficit')

print(merged_df)

In [ ]:
# Create a line chart showing revenue and expenditure trends over time
if 'year' in merged_df.columns and 'revenue' in merged_df.columns and 'expenditure' in merged_df.columns:
    fig_trend = px.line(merged_df, 
                        x='year', 
                        y=['revenue', 'expenditure'], 
                        title="Revenue vs Expenditure Trends",
                        labels={'value': 'Amount (ARS)', 'variable': 'Type'},
                        markers=True)

    fig_trend.update_layout(
        yaxis_title="Amount (ARS)",
        xaxis_title="Year",
        legend_title="Type"
    )

    fig_trend.show()
else:
    print("Required columns for trend analysis not found")

In [ ]:
# Create a bar chart showing revenue and expenditure comparison
if 'year' in merged_df.columns and 'revenue' in merged_df.columns and 'expenditure' in merged_df.columns:
    fig_bar = px.bar(merged_df, 
                     x='year', 
                     y=['revenue', 'expenditure'], 
                     title="Revenue vs Expenditure Comparison",
                     barmode='group',
                     labels={'value': 'Amount (ARS)', 'variable': 'Type'},
                     color_discrete_map={'revenue': 'blue', 'expenditure': 'red'})

    # Add value labels on bars
    fig_bar.update_traces(texttemplate='%{value:,.0f}', textposition='outside')

    fig_bar.update_layout(
        yaxis_title="Amount (ARS)",
        xaxis_title="Year",
        uniformtext_minsize=8,
        uniformtext_mode='hide'
    )

    fig_bar.show()
else:
    print("Required columns for bar chart not found")

In [ ]:
# Create a bar chart showing surplus/deficit
if 'year' in merged_df.columns and 'balance' in merged_df.columns:
    fig_balance = px.bar(merged_df, 
                         x='year', 
                         y='balance',
                         color='surplus_deficit',
                         color_discrete_map={'Surplus': 'green', 'Deficit': 'red'},
                         title="Annual Surplus/Deficit",
                         labels={'balance': 'Amount (ARS)', 'surplus_deficit': 'Status'})

    # Add value labels on bars
    fig_balance.update_traces(texttemplate='%{y:,.0f}', textposition='outside')

    fig_balance.update_layout(
        yaxis_title="Amount (ARS)",
        xaxis_title="Year",
        uniformtext_minsize=8,
        uniformtext_mode='hide'
    )

    fig_balance.show()
else:
    print("Required columns for balance chart not found")

In [ ]:
# Calculate revenue and expenditure as percentage of total
if 'balance' in merged_df.columns:
    merged_df['total_revenue'] = merged_df['revenue'].sum()
    merged_df['total_expenditure'] = merged_df['expenditure'].sum()
    merged_df['revenue_percent'] = (merged_df['revenue'] / merged_df['total_revenue']) * 100
    merged_df['expenditure_percent'] = (merged_df['expenditure'] / merged_df['total_expenditure']) * 100

    # Create subplots for percentage analysis
    fig_percentage = go.Figure()

    fig_percentage.add_trace(go.Bar(
        x=merged_df['year'],
        y=merged_df['revenue_percent'],
        name='Revenue %',
        marker_color='blue'
    ))

    fig_percentage.add_trace(go.Bar(
        x=merged_df['year'],
        y=merged_df['expenditure_percent'],
        name='Expenditure %',
        marker_color='red'
    ))

    fig_percentage.update_layout(
        title='Revenue and Expenditure as Percentage of Total',
        xaxis_title='Year',
        yaxis_title='Percentage of Total (%)',
        barmode='group'
    )

    fig_percentage.show()
else:
    print("Required columns for percentage analysis not found")

In [ ]:
# Create a doughnut chart for overall revenue vs expenditure
if 'revenue' in merged_df.columns and 'expenditure' in merged_df.columns:
    fig_doughnut = go.Figure(data=[go.Pie(
        labels=['Revenue', 'Expenditure'],
        values=[merged_df['revenue'].sum(), merged_df['expenditure'].sum()],
        hole=.4,
        title="Overall Revenue vs Expenditure",
        titleposition="middle center",
        hovertemplate='%{label}: %{value:,.0f} ARS (%{percent})<extra></extra>',
        texttemplate='%{label}: %{value:,.0f} ARS (%{percent})',
        textposition='inside'
    )])

    fig_doughnut.update_layout(
        title="Revenue vs Expenditure Distribution",
        showlegend=True,
        width=600,
        height=600
    )

    fig_doughnut.show()

    # Summary statistics
    print("Summary Statistics:")
    print(f"- Total Revenue: {merged_df['revenue'].sum():,.2f} ARS")
    print(f"- Total Expenditure: {merged_df['expenditure'].sum():,.2f} ARS")
    print(f"- Net Balance: {merged_df['balance'].sum():,.2f} ARS")
    print(f"- Average Annual Revenue: {merged_df['revenue'].mean():,.2f} ARS")
    print(f"- Average Annual Expenditure: {merged_df['expenditure'].mean():,.2f} ARS")
    
    # Deficit/Surplus analysis
    deficit_years = merged_df[merged_df['balance'] < 0]['year'].tolist()
    surplus_years = merged_df[merged_df['balance'] > 0]['year'].tolist()
    
    if deficit_years:
        print(f"- Deficit Years: {', '.join(map(str, deficit_years))}")
    if surplus_years:
        print(f"- Surplus Years: {', '.join(map(str, surplus_years))}")
else:
    print("Required columns for doughnut chart not found")